In [1]:
import datetime
import pandas as pd
import numpy as np
import math, sys, os
import scipy.stats

filename = 'data/dataset-labeled-2.csv'
dataset = pd.read_csv(filename)
print('data loaded')

data loaded


In [ ]:
dataset[dataset.prev_0>0]['prev_0'].quantile(0.25)


In [5]:
for i in [60,30,20,10,5]:
    field = 'pos_{}'.format(i)
    print("{}_q{}={}".format(field,25,dataset[field].quantile(0.25)))
    print("{}_q{}={}".format(field,50,dataset[field].quantile(0.50)))
    print("{}_q{}={}".format(field,75,dataset[field].quantile(0.75)))
    print(" "*50)

pos_60_q25=16.56
pos_60_q50=44.83
pos_60_q75=76.79
                                                  
pos_30_q25=15.85
pos_30_q50=47.27
pos_30_q75=80.97
                                                  
pos_20_q25=15.0
pos_20_q50=49.12
pos_20_q75=83.64
                                                  
pos_10_q25=11.9
pos_10_q50=50.38
pos_10_q75=89.29
                                                  
pos_5_q25=0.0
pos_5_q50=50.98
pos_5_q75=100.0
                                                  


In [ ]:
q="(trend_60==0) & (trend_30==0) & (trend_20==0) & (trend_10==0) & (trend_5==0) & (trend_3==0) & (amp_60<22.65) & (amp_10<5.98) & (prev_3>0) & (prev_2<=0) & (prev_1<0 & prev_1>=-1.59) & (prev_0>1.52)"
dataset[dataset.eval(q)].shape

In [10]:
dataset['amp_30'].quantile(0.50)

20.33